In [1]:
import pandas as pd # package for high-performance, easy-to-use data structures and data analysis
import numpy as np # fundamental package for scientific computing with Python
import matplotlib
import matplotlib.pyplot as plt # for plotting
import seaborn as sns # for making plots with seaborn
color = sns.color_palette()

from numpy import array
from matplotlib import cm
# Supress unnecessary warnings so that presentation looks clean
import warnings
warnings.filterwarnings("ignore")

from sklearn.externals import joblib

# Print all rows and columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
train = joblib.load('train1.pkl')    # 载入数据

In [6]:
listing_info = joblib.load('listing_info1.pkl')    # 载入数据

In [8]:
user_repay_logs = joblib.load('user_repay_logs1.pkl')    # 载入数据

In [ ]:
user_behavior_logs = joblib.load('user_behavior_logs1.pkl')    # 载入数据

In [9]:
user_info = joblib.load('user_info1.pkl')    # 载入数据

In [10]:
user_taglist = joblib.load('user_taglist1.pkl')    # 载入数据

In [11]:
test = joblib.load('test1.pkl')    # 载入数据

In [12]:
user_repay_logs['due_date'] = pd.to_datetime(user_repay_logs.due_date,format = '%Y-%m-%d')
user_repay_logs['repay_date'] = pd.to_datetime(user_repay_logs.repay_date,format = '%Y-%m-%d')

In [50]:
user_repay_logs['month'] = user_repay_logs['due_date'].map(lambda x:x.month)
user_repay_logs['year'] = user_repay_logs['due_date'].map(lambda x:x.year)
user_repay_logs['day'] = user_repay_logs['due_date'].map(lambda x:x.day)

In [15]:
user_repay_logs['delta'] = user_repay_logs['month'] - user_repay_logs['order_id']

In [61]:
user_repay_logs.loc[user_repay_logs['delta'] <= 0,'month'] = user_repay_logs.loc[user_repay_logs['delta'] <= 0,'delta'].map(lambda x: x+12) 
user_repay_logs.loc[user_repay_logs['delta'] <= 0,'year'] = user_repay_logs.loc[user_repay_logs['delta'] <= 0,'year'].map(lambda x: x-1)

In [63]:
user_repay_logs.loc[user_repay_logs['delta'] > 0,'month']= user_repay_logs.loc[user_repay_logs['delta'] > 0,'delta']
user_repay_logs.loc[user_repay_logs['delta'] > 0,'year'] = user_repay_logs.loc[user_repay_logs['delta'] > 0,'year']

In [65]:
user_repay_logs['year'] =  [int(i) for i in user_repay_logs['year']]
user_repay_logs['year'] = user_repay_logs['year'].astype(str)
user_repay_logs['month'] =  [int(i) for i in user_repay_logs['month']]
user_repay_logs['month'] = user_repay_logs['month'].astype(str)
user_repay_logs['day'] = user_repay_logs['day'].astype(str)

In [67]:
user_repay_logs['auditing_date'] = user_repay_logs['year'].str.cat([user_repay_logs['month'],user_repay_logs['day']],sep='-')

In [68]:
user_repay_logs['auditing_date'] = pd.to_datetime(user_repay_logs.auditing_date,format = '%Y-%m-%d')

In [70]:
user_repay_logs1 = user_repay_logs.copy()

In [72]:
user_repay_logs1.drop(['order_id','month','year','day','delta'],axis=1,inplace=True)

In [75]:
train.head()

,user_id,listing_id,auditing_date,due_date,due_amt,repay_date,repay_amt
0,748147,3163926,2018-04-25,2018-05-25,72.1167,2018-05-25,72.1167
1,672952,3698760,2018-06-09,2018-07-09,258.7045,2018-07-08,258.7045
2,404196,2355665,2018-02-18,2018-03-18,307.9270,\N,\N
3,342769,1994522,2018-01-13,2018-02-13,252.9809,2018-02-13,252.9809
4,828139,3602352,2018-06-01,2018-07-01,107.6503,2018-06-25,107.6503


In [ ]:
train['due_date'] = pd.to_datetime(train.due_date,format = '%Y-%m-%d')
train['repay_date'] = pd.to_datetime(train.repay_date,format = '%Y-%m-%d')
train['auditing_date'] = pd.to_datetime(train.auditing_date,format = '%Y-%m-%d')

In [78]:
user_repay_logs1.head()

,user_id,listing_id,auditing_date,due_date,due_amt,repay_date,repay_amt
0,748483,1858122,2017-12-29,2018-06-29,528.6365,2018-06-20,528.6365
1,748483,1858122,2017-12-29,2018-04-29,528.6365,2200-01-01,528.6365
2,748483,1858122,2017-12-29,2018-07-29,528.6365,2018-06-20,528.6365
3,748483,1858122,2017-12-29,2018-05-29,528.6365,2018-05-29,528.6365
4,748483,1858122,2017-12-29,2018-01-29,528.6365,2018-01-28,528.6365


In [77]:
order = ['user_id','listing_id','auditing_date','due_date','due_amt','repay_date','repay_amt']
user_repay_logs1 = user_repay_logs1[order]

In [ ]:
total = pd.concat([user_repay_logs1,train],axis=0)
print(total.shape)
total.drop_duplicates(keep='first',inplace=True)
print(total.shape)     # 检验是否删除了重复的记录

In [ ]:
joblib.dump(total, 'total.pkl')    # 存储合并的训练集

In [84]:
df = pd.concat([total, test], axis=0, ignore_index=True)      # 合并训练集和测试集

In [ ]:
df1 = df.merge(listing_info,on='listing_id',how='left')   # 合并标的资产信息

In [ ]:
df1['repay_date'] = df1['repay_date'].apply(
    lambda x: x['repay_date'] if x['repay_date'] != '\\N' else '2200-01-01', axis=1)
df1['repay_amt'] = df1['repay_amt'].apply(
    lambda x: x if x != '\\N' else x['due_amt'])    # 把train里为\N的替换掉

In [ ]:
df1['days']=(df1['due_date']-df1['repay_date']).apply(lambda x:x.days)   
# 提前还款天数
df1['days']=df1['days'].apply(lambda x: x if x>=0 else -1 )

In [ ]:
df1['days1']=(df1['due_date']-df1['auditing_date']).apply(lambda x:x.days)
# 应还款日期与交易日间隔天数